In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import copy
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.preprocessing import StandardScaler
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

### Read from MIMIC-IV csv files

In [3]:
csv_path = '/1MIMIC'

In [5]:
with open(csv_path+'/CSV/diagnoses_icd.csv/diagnoses_icd.csv', 'rb') as f:
    diagnoses_icd = pd.read_csv(f)
with open(csv_path+'/CSV/poe.csv/poe.csv', 'rb') as f:
    df_poe = pd.read_csv(f)
with open(csv_path+'/CSV/d_icd_diagnoses.csv/d_icd_diagnoses.csv', 'rb') as f:
    diagnose_dic = pd.read_csv(f)
with open(csv_path+'/CSV/labevents.csv/labevents.csv', 'rb') as f:
    lab_df = pd.read_csv(f)
with open(csv_path+'/CSV/omr.csv/omr.csv', 'rb') as f:
    omr_df = pd.read_csv(f)
with open(csv_path+'/CSV/admissions.csv/admissions.csv', 'rb') as f:
    adm_df = pd.read_csv(f)
with open(csv_path+'/CSV/patients.csv/patients.csv', 'rb') as f:
    patient_df = pd.read_csv(f)

---
### Process data
#### Laboratory examinations

In [6]:
# extract all lab records
lab_all=lab_df[['itemid','subject_id','hadm_id','valuenum','charttime']]

In [7]:
# drop duplicate records for the same lab item in one time of visit, keep the first one  
lab_all=lab_all.sort_values('charttime',ascending=True).drop_duplicates(['hadm_id','itemid'],keep='first')

# filter lab items which have testing frequencies >=5%
labitem_cnt=lab_all.groupby('itemid').hadm_id.nunique().sort_values(ascending=False).reset_index()
labitem_cnt.rename(columns={'hadm_id':'cnt'},inplace=True)
labitem_cnt['percent']=labitem_cnt['cnt']/lab_all.hadm_id.nunique()*100
labitem_list=labitem_cnt[labitem_cnt.percent>=5].itemid
lab_all=lab_all[lab_all.itemid.isin(labitem_list)]

In [8]:
# reshaping the long table into a wide table by making lab items as columns 
lab_all = pd.pivot_table(lab_all, values='valuenum', index=['subject_id', 'hadm_id'], columns='itemid')
# Creating a dictionary to map the existing column names to new names prefixed with 'M'
new_column_names = {col: 'M' + str(col) for col in lab_all.columns}
# Renaming the columns of the 'lab_all' DataFrame using the dictionary created
lab_all = lab_all.rename(columns=new_column_names)

#### Condition

In [9]:
# create disease category - icd_code starting digits mapping dictionary 
disease_icd_mapping = {
    'ARR': ['I47','I48','I49','427'],
    'AF': ['I48','42731'],
    'CAS': ['I652','43310','43311'],
    'DM': ['E10','E11','E12','E13','E14','250'],
    'HTN': ['I10','I11','I12','I13','I15',
            '401','402','403','404','405'],
    'CM': ['I42','425'],
    'CHD': ['I20','I21','I24','I25','I70',
            '410','411','412','413','414',
           '4292','440'],
    'HLP': ['E78','272'],
    'GERD': ['K219','5301'],
    'HUA': ['E790']
}

# map disease category to exact icd-codes
for disease, icd_codes in disease_icd_mapping.items():
    _diag_list=[]
    for code in icd_codes:
        _diag_list.extend(diagnose_dic[diagnose_dic.icd_code.str.startswith(code)].icd_code.values)
    disease_icd_mapping[disease] = _diag_list
    
# create a list of target diseases' corresponding icd-codes 
target_disease=[item for sublist in disease_icd_mapping.values() for item in sublist]

In [10]:
# filter condition in target disease lists
condition_selected=diagnoses_icd[diagnoses_icd.icd_code.isin(target_disease)][['subject_id',
                                                                         'hadm_id','icd_code']]
condition_selected['value'] = 1
con_all = pd.pivot_table(condition_selected, values='value', index=['subject_id', 'hadm_id'], columns='icd_code', fill_value=np.nan)


In [11]:
for disease in target_disease:
    # Check if the disease column exists in df_cond
    if disease not in con_all.columns:
        # If not, add the column with np.nan values
        con_all[disease] = np.nan
# map icd_code to our target disease categories 
for key in disease_icd_mapping.keys():
    con_all['C_'+key]=(np.sum(con_all[disease_icd_mapping[key]], axis=1) > 0).replace(True, 1).replace(False, np.nan)
con_all=con_all[con_all.columns[con_all.columns.str.startswith('C_')].values]

#### Vital Sign

In [12]:
adm_selected=adm_df[['subject_id','hadm_id','admittime','dischtime']]
# for vital signs that have multiple records at the same dischtime, keep the first record 
omr_selected=omr_df.drop_duplicates(['subject_id', 'chartdate','result_name'],keep='first').drop('seq_num',axis=1)

# merge admission detail and vital signs into a single dataframe merged_df
merged_df = pd.merge(adm_selected, omr_selected, on='subject_id')

# filter vital signs before admission time of the current visit 
filtered_df = merged_df[merged_df['chartdate'] < merged_df['admittime']]

# Sort by subject_id, admittime, and chart_date
sorted_df = filtered_df.sort_values(['subject_id', 'hadm_id','admittime', 'chartdate'])

# reshaping the long table into a wide table by making vital signs as columns 
omr_selected = pd.pivot_table(sorted_df, values='result_value', index=['subject_id', 'hadm_id','admittime', 'dischtime'], columns='result_name', aggfunc='last').reset_index()

In [13]:
# merge with patient.csv to calculate patient age
omr_selected=omr_selected.merge(patient_df[['subject_id','anchor_age','anchor_year']],on='subject_id',how='left')

#Creating a datetime object for January 1, 00:00:00 of the given year
omr_selected['anchor_year']=omr_selected.anchor_year.apply(lambda x:datetime(x, 1, 1, 0, 0, 0))
#Change to datetime object
omr_selected.admittime=omr_selected.admittime.apply(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
# age=admittime- anchor_year + anchor_age
omr_selected['age']=((omr_selected.admittime-omr_selected.anchor_year).dt.days / 365) + omr_selected.anchor_age


In [14]:
omr_selected=omr_selected.set_index(['subject_id', 'hadm_id'])
# Drop columns with more than 90% NAs
omr_cols=omr_selected.columns[ omr_selected.isna().sum()/omr_selected.shape[0]<0.9]
omr_selected=omr_selected[omr_cols]

# split blood pressure and store them in two columns 
omr_selected[['Blood_Pressure_High', 'Blood_Pressure_Low']] = omr_selected['Blood Pressure'].str.split('/', expand=True)
omr_selected.drop(['admittime','anchor_age','anchor_year','Blood Pressure', 'dischtime'],axis=1,inplace=True)

#### assign provider

In [15]:
# Filtering rows in df_poe where the order_subtype is 'Discharge' and selecting specific columns
dis_provider_df = df_poe[df_poe.order_subtype == 'Discharge'][['subject_id', 'hadm_id', 'order_provider_id', 'ordertime']]

# Sorting the dis_provider_df DataFrame based on the 'ordertime' column and keeping only the last hadm_id's corresponding order_provider_id
dis_provider_df = dis_provider_df.sort_values('ordertime').drop_duplicates('hadm_id', keep='last')

# Dropping the 'ordertime' column from dis_provider_df
dis_provider_df.drop('ordertime', axis=1, inplace=True)

#### Merge all datasets

In [16]:
# Joining lab results, diagnoses, vital signs and provider information into a single DataFrame df_all
df_all = lab_all.join(con_all).join(omr_selected).join(dis_provider_df.set_index(['subject_id', 'hadm_id'])).reset_index()

In [17]:
# Count the number of unique hadm_ids per order_provider_id
count_df = df_all.groupby('order_provider_id')['hadm_id'].nunique().reset_index().rename(columns={'hadm_id':'visits'}).sort_values(by=['visits'], ascending=False)

# Sort the DataFrame based on the count in descending order
sorted_df = df_all.merge(count_df, on='order_provider_id', how='left').sort_values(by=['visits', 'order_provider_id', 'subject_id', 'hadm_id'], ascending=False)
df_all_indexed = sorted_df.set_index(['order_provider_id','subject_id','hadm_id']).drop('visits',axis=1)

---
### Extracting top physicians

#### set number of folds and physicians

In [18]:
n_folds, n_providers_per_fold = 11, 5
n_providers = n_folds * n_providers_per_fold 
top_providers = count_df.iloc[0:n_providers]['order_provider_id']

In [19]:
# split label and features to two dataframes 
df_label = df_all_indexed.loc[top_providers][df_all_indexed.columns[df_all_indexed.columns.str.startswith('C_')]]
df_feature = df_all_indexed.loc[top_providers][df_all_indexed.columns[~df_all_indexed.columns.str.startswith('C_')]]

#### Normalization

In [20]:
# normalization
df_feature_scaled = df_feature.copy()
for col in df_feature:
    df_feature_scaled[col] = StandardScaler().fit_transform(df_feature[col].values.reshape(-1, 1))

#### storage

In [21]:
with open(os.getcwd()+'/data/mimic/df_all.pickle', 'wb') as f:
     pickle.dump(df_all_indexed, f)
with open(os.getcwd()+'/data/mimic/df_feature.pickle', 'wb') as f:
     pickle.dump(df_feature_scaled, f)
with open(os.getcwd()+'/data/mimic/df_label.pickle', 'wb') as f:
     pickle.dump(df_label, f)